<a href="https://colab.research.google.com/github/rizkaaa19/thesis/blob/main/tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

full_df = pd.read_excel("/content/cpdg.xlsx")
full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

full_df = (
    full_df
    .sort_values('timestamp')
    .drop_duplicates(subset='timestamp')
    .set_index('timestamp')
)

full_df.columns = (
    full_df.columns
    .str.strip()
    .str.replace(' ', '', regex=False)
)

COL_E = 'Easting[m]'
COL_N = 'Northing[m]'
COL_H = 'Ortho.Height[m]'

full_df[COL_E] /= 1000.0
full_df[COL_N] /= 1000.0
full_df[COL_H] /= 1000.0

print("Kolom tersedia:")
print(full_df.columns.tolist())

print("\nPreview data (E, N, H):")
print(full_df[[COL_E, COL_N, COL_H]].head())

Kolom tersedia:
['Easting[m]', 'Northing[m]', 'Ortho.Height[m]']

Preview data (E, N, H):
                      Easting[m]   Northing[m]  Ortho.Height[m]
timestamp                                                      
2021-01-01 06:59:42  6516739.400  9.894526e+07          131.292
2021-01-02 06:59:42  6516737.334  9.894526e+07          132.741
2021-01-03 06:59:42  6516739.724  9.894526e+07          128.494
2021-01-04 06:59:42  6516739.993  9.894526e+07          129.754
2021-01-05 06:59:42  6516738.984  9.894526e+07          128.723


In [ ]:
geo_df = full_df[[COL_E, COL_N, COL_H]]

delta_df = geo_df.diff().dropna()
delta_df.columns = ['dE', 'dN', 'dH']

print(delta_df.head())

                        dE     dN     dH
timestamp                               
2021-01-02 06:59:42 -2.066 -2.023  1.449
2021-01-03 06:59:42  2.390  0.318 -4.247
2021-01-04 06:59:42  0.269  2.343  1.260
2021-01-05 06:59:42 -1.009  0.001 -1.031
2021-01-06 06:59:42  1.657 -2.112  1.489


In [ ]:
WINDOW_SMOOTH = 7

delta_smooth = delta_df.rolling(
    window=WINDOW_SMOOTH,
    center=False
).mean().dropna()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_E = MinMaxScaler(feature_range=(-1, 1))
scaler_N = MinMaxScaler(feature_range=(-1, 1))
scaler_H = MinMaxScaler(feature_range=(-1, 1))

dE_scaled = scaler_E.fit_transform(delta_smooth[['dE']])
dN_scaled = scaler_N.fit_transform(delta_smooth[['dN']])
dH_scaled = scaler_H.fit_transform(delta_smooth[['dH']])

scaled_data = np.hstack([dE_scaled, dN_scaled, dH_scaled])

In [ ]:
TIME_STEPS = 30

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, TIME_STEPS)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate

input_layer = Input(shape=(TIME_STEPS, 3))

# Branch E
branch_E = Lambda(lambda x: x[:, :, 0:1])(input_layer)
branch_E = LSTM(32, return_sequences=False)(branch_E)

# Branch N
branch_N = Lambda(lambda x: x[:, :, 1:2])(input_layer)
branch_N = LSTM(32, return_sequences=False)(branch_N)

# Branch H
branch_H = Lambda(lambda x: x[:, :, 2:3])(input_layer)
branch_H = LSTM(32, return_sequences=False)(branch_H)

# Gabung
merged = Concatenate()([branch_E, branch_N, branch_H])
dense = Dense(32, activation='relu')(merged)

output = Dense(3, name='Predicted_dENH')(dense)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 30, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 30, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 30, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │      4,352 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │      4,352 │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 32)        │      4,352 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm_1[0][0],     │
│                     │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      3,104 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Predicted_dENH      │ (None, 3)         │         99 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,259 (63.51 KB)

 Trainable params: 16,259 (63.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
n = len(X)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

y_pred = model.predict(X_test)

dE_true = scaler_E.inverse_transform(y_test[:, 0:1])
dN_true = scaler_N.inverse_transform(y_test[:, 1:2])
dH_true = scaler_H.inverse_transform(y_test[:, 2:3])

dE_pred = scaler_E.inverse_transform(y_pred[:, 0:1])
dN_pred = scaler_N.inverse_transform(y_pred[:, 1:2])
dH_pred = scaler_H.inverse_transform(y_pred[:, 2:3])

def metrics(name, y_true, y_hat):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_hat)
    print(f"\n[{name}]")
    print("MSE :", mse)
    print("RMSE:", rmse)
    print("MAE :", mae)

metrics("dE (m/hari)", dE_true, dE_pred)
metrics("dN (m/hari)", dN_true, dN_pred)
metrics("dH (m/hari)", dH_true, dH_pred)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - loss: 0.0238 - val_loss: 0.0137
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0170 - val_loss: 0.0135
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0165 - val_loss: 0.0136
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0151 - val_loss: 0.0132
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0169 - val_loss: 0.0132
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0144 - val_loss: 0.0129
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0115 - val_loss: 0.0128
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0136 - val_loss: 0.0134
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0143 - val_loss: 0.0130
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0132 - val_loss: 0.0129
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0128 - val_loss: 0.0129
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.

In [ ]:
full_df = pd.read_excel("/content/cpdg.xlsx")

full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

full_df = (
    full_df
    .sort_values('timestamp')
    .drop_duplicates(subset='timestamp')
    .set_index('timestamp')
)

full_df.columns = (
    full_df.columns
    .str.strip()
    .str.replace(' ', '', regex=False)
)

COL_E = 'Easting[m]'
COL_N = 'Northing[m]'
COL_H = 'Ortho.Height[m]'

full_df[COL_E] = full_df[COL_E] / 1000.0
full_df[COL_N] = full_df[COL_N] / 1000.0

full_df[COL_H] = full_df[COL_H] / 1000.0

print("Kolom tersedia:", full_df.columns.tolist())
print("\nPreview data (E, N, H):")
print(full_df[[COL_E, COL_N, COL_H]].head())

Kolom tersedia: ['Easting[m]', 'Northing[m]', 'Ortho.Height[m]']

Preview data (E, N, H):
                      Easting[m]   Northing[m]  Ortho.Height[m]
timestamp                                                      
2021-01-01 06:59:42  6516739.400  9.894526e+07          131.292
2021-01-02 06:59:42  6516737.334  9.894526e+07          132.741
2021-01-03 06:59:42  6516739.724  9.894526e+07          128.494
2021-01-04 06:59:42  6516739.993  9.894526e+07          129.754
2021-01-05 06:59:42  6516738.984  9.894526e+07          128.723


In [ ]:
monthly_pos = full_df[[COL_E, COL_N, COL_H]].resample('MS').median()

In [ ]:
monthly_delta = monthly_pos.diff().dropna()
monthly_delta.columns = ['dE_m', 'dN_m', 'dH_m']

In [ ]:
H_month = monthly_pos[COL_H].copy()

H_trend = H_month.rolling(window=5, center=True, min_periods=1).median()

dH_trend = H_trend.diff().dropna()

monthly_delta2 = monthly_delta.copy()
monthly_delta2['dH_m'] = dH_trend

In [ ]:
monthly_delta2['dH_m'] = (
    monthly_delta2['dH_m']
    .rolling(window=3, center=True, min_periods=1)
    .mean()
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

features = monthly_delta2[['dE_m', 'dN_m', 'dH_m']].copy()

scaler_E = MinMaxScaler(feature_range=(-1, 1))
scaler_N = MinMaxScaler(feature_range=(-1, 1))
scaler_H = MinMaxScaler(feature_range=(-1, 1))

dE_scaled = scaler_E.fit_transform(features[['dE_m']])
dN_scaled = scaler_N.fit_transform(features[['dN_m']])
dH_scaled = scaler_H.fit_transform(features[['dH_m']])

scaled_monthly = np.hstack([dE_scaled, dN_scaled, dH_scaled])

In [ ]:
import numpy as np

TIME_STEPS = 6

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_monthly, TIME_STEPS)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (41, 6, 3)
y shape: (41, 3)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate

input_layer = Input(shape=(TIME_STEPS, 3), name="Input_6Bulan")

# Branch E
branch_E = Lambda(lambda x: x[:, :, 0:1], name="Select_dE")(input_layer)
branch_E = LSTM(32, return_sequences=False, name="LSTM_dE")(branch_E)

# Branch N
branch_N = Lambda(lambda x: x[:, :, 1:2], name="Select_dN")(input_layer)
branch_N = LSTM(32, return_sequences=False, name="LSTM_dN")(branch_N)

# Branch H
branch_H = Lambda(lambda x: x[:, :, 2:3], name="Select_dH")(input_layer)
branch_H = LSTM(32, return_sequences=False, name="LSTM_dH")(branch_H)

# Gabung semua branch
merged = Concatenate(name="Merge_ENH")([branch_E, branch_N, branch_H])

# Dense
dense = Dense(32, activation="relu", name="Dense_After_Merge")(merged)

# Output: 1 nilai (prediksi dH bulan depan, masih skala -1..1)
output = Dense(1, name="Predicted_dH_next_month")(dense)

# Build & compile
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss="mse")

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_6Bulan        │ (None, 6, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dE (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dN (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dH (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dE (LSTM)      │ (None, 32)        │      4,352 │ Select_dE[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dN (LSTM)      │ (None, 32)        │      4,352 │ Select_dN[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dH (LSTM)      │ (None, 32)        │      4,352 │ Select_dH[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Merge_ENH           │ (None, 96)        │          0 │ LSTM_dE[0][0],    │
│ (Concatenate)       │                   │            │ LSTM_dN[0][0],    │
│                     │                   │            │ LSTM_dH[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_After_Merge   │ (None, 32)        │      3,104 │ Merge_ENH[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Predicted_dH_next_… │ (None, 1)         │         33 │ Dense_After_Merg… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,193 (63.25 KB)

 Trainable params: 16,193 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
y_dH = y[:, 2:3]

In [ ]:
n = len(X)

train_end = int(0.7 * n)
val_end   = int(0.85 * n)

X_train, y_train = X[:train_end], y_dH[:train_end]
X_val,   y_val   = X[train_end:val_end], y_dH[train_end:val_end]
X_test,  y_test  = X[val_end:], y_dH[val_end:]

print("Total sampel:", n)
print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)

Total sampel: 41
Train: (28, 6, 3) (28, 1)
Val  : (6, 6, 3) (6, 1)
Test : (7, 6, 3) (7, 1)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    batch_size=8,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 243ms/step - loss: 0.2483 - val_loss: 0.4132
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2335 - val_loss: 0.3756
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2284 - val_loss: 0.3365
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2279 - val_loss: 0.2983
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1732 - val_loss: 0.2669
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1825 - val_loss: 0.2406
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1585 - val_loss: 0.2262
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1674 - val_loss: 0.2177
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2236 - val_loss: 0.2277
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1799 - val_loss: 0.2271
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1294 - val_loss: 0.2143
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1403 - val_l

In [ ]:
import numpy as np

y_pred = model.predict(X_test, verbose=0)

dH_true_m = scaler_H.inverse_transform(y_test)
dH_pred_m = scaler_H.inverse_transform(y_pred)

dH_true_mm = dH_true_m * 1000.0
dH_pred_mm = dH_pred_m * 1000.0

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(dH_true_mm, dH_pred_mm)
rmse = np.sqrt(mse)
mae = mean_absolute_error(dH_true_mm, dH_pred_mm)

In [ ]:
result_df = pd.DataFrame({
    "dH_true_mm": dH_true_mm.ravel(),
    "dH_pred_mm": dH_pred_mm.ravel(),
    "error_mm": (dH_pred_mm - dH_true_mm).ravel()
})

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# LOAD DATA
FILE_XLSX = "/content/cpdg.xlsx"
SHEET_NAME = 0
TIME_COL = "timestamp"

COL_E = "Easting [m]"
COL_N = "Northing [m]"
COL_H = "Ortho. Height [m]"

df = pd.read_excel(FILE_XLSX, sheet_name=SHEET_NAME)
df.columns = df.columns.astype(str).str.strip()

if TIME_COL not in df.columns:
    raise ValueError(f"Kolom waktu '{TIME_COL}' tidak ditemukan. Kolom yang ada: {df.columns.tolist()}")

df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
df = df.dropna(subset=[TIME_COL])

for c in [COL_E, COL_N, COL_H]:
    if c not in df.columns:
        raise ValueError(f"Kolom '{c}' tidak ditemukan. Kolom yang ada: {df.columns.tolist()}")

df[COL_E] = pd.to_numeric(df[COL_E], errors="coerce")
df[COL_N] = pd.to_numeric(df[COL_N], errors="coerce")
df[COL_H] = pd.to_numeric(df[COL_H], errors="coerce")
df = df.dropna(subset=[COL_E, COL_N, COL_H])

df = df.sort_values(TIME_COL).drop_duplicates(subset=[TIME_COL]).set_index(TIME_COL)

print("SEBELUM konversi unit:")
print(df[[COL_E, COL_N, COL_H]].head())

# Easting/Northing
if df[COL_E].median() > 1e7:
    df[COL_E] = df[COL_E] / 1000.0
if df[COL_N].median() > 1e7:
    df[COL_N] = df[COL_N] / 1000.0

# Height
if df[COL_H].median() > 1000:
    df[COL_H] = df[COL_H] / 1000.0

print("SESUDAH konversi unit:")
print(df[[COL_E, COL_N, COL_H]].head())
print("\nOK. Data siap:", df.shape)

# MONTHLY + DELTA
monthly_pos = df[[COL_E, COL_N, COL_H]].resample("MS").median()

monthly_delta = monthly_pos.diff().dropna()
monthly_delta.columns = ["dE_m", "dN_m", "dH_m"]  # meter/bulan

# REDUKSI NOISE
H_month = monthly_pos[COL_H].copy()
H_trend = H_month.rolling(window=5, center=True, min_periods=1).median()
dH_trend = H_trend.diff().dropna()

monthly_delta2 = monthly_delta.copy()
monthly_delta2["dH_m"] = dH_trend
monthly_delta2["dH_m"] = monthly_delta2["dH_m"].rolling(window=3, center=True, min_periods=1).mean()

# SCALING (per-branch)
scaler_E = MinMaxScaler(feature_range=(-1, 1))
scaler_N = MinMaxScaler(feature_range=(-1, 1))
scaler_H = MinMaxScaler(feature_range=(-1, 1))

dE_scaled = scaler_E.fit_transform(monthly_delta2[["dE_m"]])
dN_scaled = scaler_N.fit_transform(monthly_delta2[["dN_m"]])
dH_scaled = scaler_H.fit_transform(monthly_delta2[["dH_m"]])

scaled_monthly = np.hstack([dE_scaled, dN_scaled, dH_scaled])

# WINDOWING
TIME_STEPS = 6

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_monthly, TIME_STEPS)
y_dH = y[:, 2:3]  # target hanya dH (scaled)
target_months = monthly_delta2.index[TIME_STEPS:]

# SPLIT TRAIN/VAL/TEST
n = len(X)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y_dH[:train_end]
X_val, y_val     = X[train_end:val_end], y_dH[train_end:val_end]
X_test, y_test   = X[val_end:], y_dH[val_end:]
test_months      = target_months[val_end:]

print("Train/Val/Test:", X_train.shape, X_val.shape, X_test.shape)

# PARALLEL LSTM (3 cabang)
input_layer = Input(shape=(TIME_STEPS, 3), name="Input_6Bulan")

branch_E = Lambda(lambda x: x[:, :, 0:1], name="Select_dE")(input_layer)
branch_E = LSTM(32, return_sequences=False, name="LSTM_dE")(branch_E)

branch_N = Lambda(lambda x: x[:, :, 1:2], name="Select_dN")(input_layer)
branch_N = LSTM(32, return_sequences=False, name="LSTM_dN")(branch_N)

branch_H = Lambda(lambda x: x[:, :, 2:3], name="Select_dH")(input_layer)
branch_H = LSTM(32, return_sequences=False, name="LSTM_dH")(branch_H)

merged = Concatenate(name="Merge_ENH")([branch_E, branch_N, branch_H])
dense  = Dense(32, activation="relu", name="Dense_After_Merge")(merged)
output = Dense(1, name="Predicted_dH_next_month")(dense)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss=tf.keras.losses.Huber(delta=0.2))
model.summary()

# TRAIN
early_stop = EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    batch_size=8,
    callbacks=[early_stop],
    verbose=1
)

# TEST EVAL
y_pred_test = model.predict(X_test, verbose=0)

dH_true_m_test = scaler_H.inverse_transform(y_test).ravel()
dH_pred_m_test = scaler_H.inverse_transform(y_pred_test).ravel()

mse  = mean_squared_error(dH_true_m_test, dH_pred_m_test)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(dH_true_m_test, dH_pred_m_test)

eval_table = pd.DataFrame({"Model": ["PLSTM"], "MSE": [mse], "RMSE": [rmse], "MAE": [mae]})
print("\n=== TABEL EVALUASI ===")
print(eval_table)

# TABEL TEST
first_test_month = pd.Timestamp(test_months[0])
baseline_month = first_test_month - pd.offsets.MonthBegin(1)

if baseline_month in monthly_pos.index:
    H0_abs = float(monthly_pos.loc[baseline_month, COL_H])
else:
    prev_idx = monthly_pos.index[monthly_pos.index < baseline_month]
    if len(prev_idx) == 0:
        raise ValueError("Tidak ditemukan baseline H sebelum data test.")
    H0_abs = float(monthly_pos.loc[prev_idx[-1], COL_H])

H_obs_abs = H0_abs + np.cumsum(dH_true_m_test)
H_pred_abs = H0_abs + np.cumsum(dH_pred_m_test)

subs_obs_m = -(H_obs_abs - H0_abs)
subs_pred_m = -(H_pred_abs - H0_abs)

amp_test = np.nanmax(subs_obs_m) - np.nanmin(subs_obs_m)
scale_m_test = 1.0 if amp_test == 0 else amp_test

obs_series = 2.0 + (subs_obs_m / scale_m_test)
pred_series = 2.0 + (subs_pred_m / scale_m_test)
label_unit = "SubsidenE"

result_level_test = pd.DataFrame({"timestamp": test_months, "Observation": obs_series, "Prediction": pred_series}).set_index("timestamp")
result_level_test["Year"] = result_level_test.index.year
result_level_test["Month"] = result_level_test.index.strftime("%B")

month_order = ["January","February","March","April","May","June","July","August","September","October","November","December"]

obs_pivot = result_level_test.pivot_table(index="Month", columns="Year", values="Observation", aggfunc="mean").reindex(month_order)
pred_pivot = result_level_test.pivot_table(index="Month", columns="Year", values="Prediction", aggfunc="mean").reindex(month_order)

months_to_keep = ["June","July","August","September","October","November","December"]
obs_pivot = obs_pivot.loc[months_to_keep]
pred_pivot = pred_pivot.loc[months_to_keep]

obs_pivot.loc["Total"] = obs_pivot.sum(axis=0, numeric_only=True)
pred_pivot.loc["Total"] = pred_pivot.sum(axis=0, numeric_only=True)

print(f"\n=== OBSERVATION TEST ({label_unit}) ===")
print(obs_pivot.round(2))

print(f"\n=== PREDICTION TEST ({label_unit}) ===")
print(pred_pivot.round(2))

# PREDICTION 2025–2029

H_hist = monthly_pos[COL_H].copy()
subs_hist_m = -(H_hist - H_hist.iloc[0])
p5, p95 = np.percentile(subs_hist_m.dropna().values, [5, 95])
scale_m_hist = (p95 - p5)
if scale_m_hist == 0:
    scale_m_hist = 1.0

seasonal = monthly_delta2["dH_m"].groupby(monthly_delta2.index.month).mean()
seasonal = seasonal - seasonal.mean()

if seasonal.abs().max() == 0:
    seasonal_pattern = seasonal * 0.0
else:
    seasonal_pattern = seasonal / seasonal.abs().max() * 0.12

dH_hist = monthly_delta2["dH_m"].dropna().values
low, high = np.percentile(dH_hist, [5, 95])

prediction_start = pd.Timestamp("2025-01-01")
prediction_end   = pd.Timestamp("2029-12-01")
future_months = pd.date_range(start=prediction_start, end=prediction_end, freq="MS")
num_prediction_months = len(future_months)

current_input_sequence = scaled_monthly[-TIME_STEPS:].copy()
dH_pred_m_prediction = []

for _ in range(num_prediction_months):
    inp = current_input_sequence.reshape(1, TIME_STEPS, 3)

    dH_scaled_pred_next = model.predict(inp, verbose=0)  # (1,1)
    dH_scaled_val = float(dH_scaled_pred_next.reshape(-1)[0])

    dH_m_pred_next = float(scaler_H.inverse_transform([[dH_scaled_val]])[0, 0])
    dH_m_pred_next = float(np.clip(dH_m_pred_next, low, high))
    dH_pred_m_prediction.append(dH_m_pred_next)

    last_dE_scaled = float(current_input_sequence[-1, 0])
    last_dN_scaled = float(current_input_sequence[-1, 1])

    new_step = np.array([last_dE_scaled, last_dN_scaled, dH_scaled_val], dtype=float)
    current_input_sequence = np.vstack([current_input_sequence[1:], new_step])

dH_pred_m_prediction = np.array(dH_pred_m_prediction, dtype=float)

H_start_prediction = float(monthly_pos[COL_H].iloc[-1])
H_pred_abs_prediction = H_start_prediction + np.cumsum(dH_pred_m_prediction)
subs_future_m = -(H_pred_abs_prediction - H_start_prediction)

prediction_scaled_list = []
for ts, subs_m in zip(future_months, subs_future_m):
    seas = float(seasonal_pattern.loc[ts.month]) if ts.month in seasonal_pattern.index else 0.0
    val = 2.0 + (subs_m / scale_m_hist) + seas
    prediction_scaled_list.append(val)

prediction_scaled_df = pd.DataFrame({"timestamp": future_months, "prediction_Scaled_2_3": prediction_scaled_list}).set_index("timestamp")
prediction_scaled_df["Year"] = prediction_scaled_df.index.year
prediction_scaled_df["Month"] = prediction_scaled_df.index.strftime("%B")

prediction_scaled_pivot = prediction_scaled_df.pivot_table(
    index="Month", columns="Year", values="prediction_Scaled_2_3", aggfunc="mean"
).reindex(month_order)

prediction_scaled_pivot = prediction_scaled_pivot.reindex(columns=[2025, 2026, 2027, 2028, 2029])
prediction_scaled_pivot.loc["Total"] = prediction_scaled_pivot.sum(axis=0, numeric_only=True)

print("\n=== PREDICTION (2025–2029) ===")
print(prediction_scaled_pivot.round(2))

SEBELUM konversi unit:
                     Easting [m]  Northing [m]  Ortho. Height [m]
timestamp                                                        
2021-01-01 06:59:42   6516739400   98945262132             131292
2021-01-02 06:59:42   6516737334   98945260109             132741
2021-01-03 06:59:42   6516739724   98945260427             128494
2021-01-04 06:59:42   6516739993   98945262770             129754
2021-01-05 06:59:42   6516738984   98945262771             128723
SESUDAH konversi unit:
                     Easting [m]  Northing [m]  Ortho. Height [m]
timestamp                                                        
2021-01-01 06:59:42  6516739.400  9.894526e+07            131.292
2021-01-02 06:59:42  6516737.334  9.894526e+07            132.741
2021-01-03 06:59:42  6516739.724  9.894526e+07            128.494
2021-01-04 06:59:42  6516739.993  9.894526e+07            129.754
2021-01-05 06:59:42  6516738.984  9.894526e+07            128.723

OK. Data siap: (1450, 3)
Trai

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_6Bulan        │ (None, 6, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dE (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dN (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Select_dH (Lambda)  │ (None, 6, 1)      │          0 │ Input_6Bulan[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dE (LSTM)      │ (None, 32)        │      4,352 │ Select_dE[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dN (LSTM)      │ (None, 32)        │      4,352 │ Select_dN[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM_dH (LSTM)      │ (None, 32)        │      4,352 │ Select_dH[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Merge_ENH           │ (None, 96)        │          0 │ LSTM_dE[0][0],    │
│ (Concatenate)       │                   │            │ LSTM_dN[0][0],    │
│                     │                   │            │ LSTM_dH[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_After_Merge   │ (None, 32)        │      3,104 │ Merge_ENH[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Predicted_dH_next_… │ (None, 1)         │         33 │ Dense_After_Merg… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,193 (63.25 KB)

 Trainable params: 16,193 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 253ms/step - loss: 0.0631 - val_loss: 0.0963
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0608 - val_loss: 0.0831
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0541 - val_loss: 0.0713
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0415 - val_loss: 0.0622
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0402 - val_loss: 0.0572
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0585 - val_loss: 0.0552
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0424 - val_loss: 0.0531
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0422 - val_loss: 0.0535
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0483 - val_loss: 0.0578
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0436 - val_loss: 0.0599
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0321 - val_loss: 0.0612
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0435 - val_l

In [ ]:
!pwd
!ls

/content
cpdg.xlsx  deploy  sample_data


In [ ]:
!mkdir -p deploy
!ls

cpdg.xlsx  deploy  sample_data


In [ ]:
%cd /content/deploy
!pwd
!ls

/content/deploy
/content/deploy
app.py	deploy	requirements.txt


In [ ]:
%%writefile app.py
import streamlit as st

st.set_page_config(page_title="Demo PLSTM", layout="centered")
st.title("PLSTM Subsidence App (Dummy)")
st.write("app.py berhasil dijalankan.")

Overwriting app.py


In [ ]:
!ls

app.py	deploy	requirements.txt


In [ ]:
%%writefile requirements.txt
streamlit
pandas
numpy
scikit-learn
tensorflow
openpyxl
joblib

Overwriting requirements.txt


In [ ]:
!ls

app.py	deploy	requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.187.61:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1873, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1269, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# =========================================================
# STREAMLIT UI
# =========================================================
st.set_page_config(page_title="PLSTM Land Subsidence", layout="centered")
st.title("AI-Based Land Subsidence Prediction (PLSTM)")
st.write("Upload data GNSS (Excel), lalu jalankan model.")

uploaded_file = st.file_uploader("Upload file Excel (.xlsx)", type=["xlsx"])

if uploaded_file is None:
    st.info("Silakan upload file Excel untuk memulai.")
    st.stop()

# LOAD DATA
TIME_COL = "timestamp"
COL_E = "Easting [m]"
COL_N = "Northing [m]"
COL_H = "Ortho. Height [m]"

df = pd.read_excel(uploaded_file)
df.columns = df.columns.astype(str).str.strip()

df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
df = df.dropna(subset=[TIME_COL])

df[COL_E] = pd.to_numeric(df[COL_E], errors="coerce")
df[COL_N] = pd.to_numeric(df[COL_N], errors="coerce")
df[COL_H] = pd.to_numeric(df[COL_H], errors="coerce")
df = df.dropna(subset=[COL_E, COL_N, COL_H])

df = df.sort_values(TIME_COL).set_index(TIME_COL)

# Fix unit
if df[COL_E].median() > 1e7:
    df[COL_E] /= 1000.0
if df[COL_N].median() > 1e7:
    df[COL_N] /= 1000.0
if df[COL_H].median() > 1000:
    df[COL_H] /= 1000.0

st.success("Data berhasil dibaca dan divalidasi.")

monthly_pos = df[[COL_E, COL_N, COL_H]].resample("MS").median()

monthly_delta = monthly_pos.diff().dropna()
monthly_delta.columns = ["dE_m", "dN_m", "dH_m"]

# REDUKSI NOISE
H_trend = monthly_pos[COL_H].rolling(5, center=True, min_periods=1).median()
monthly_delta["dH_m"] = H_trend.diff().dropna()
monthly_delta["dH_m"] = monthly_delta["dH_m"].rolling(3, center=True, min_periods=1).mean()

# NORMALISASI
scaler_E = MinMaxScaler((-1, 1))
scaler_N = MinMaxScaler((-1, 1))
scaler_H = MinMaxScaler((-1, 1))

dE = scaler_E.fit_transform(monthly_delta[["dE_m"]])
dN = scaler_N.fit_transform(monthly_delta[["dN_m"]])
dH = scaler_H.fit_transform(monthly_delta[["dH_m"]])

scaled = np.hstack([dE, dN, dH])

# WINDOWING
TIME_STEPS = 6

def make_seq(data, ts):
    X, y = [], []
    for i in range(len(data) - ts):
        X.append(data[i:i+ts])
        y.append(data[i+ts, 2])
    return np.array(X), np.array(y)

X, y = make_seq(scaled, TIME_STEPS)
y = y.reshape(-1, 1)

# SPLIT DATA
n = len(X)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

test_months = monthly_delta.index[TIME_STEPS + val_end:]

# PARALLEL LSTM
inp = Input(shape=(TIME_STEPS, 3))

bE = LSTM(32)(Lambda(lambda x: x[:, :, 0:1])(inp))
bN = LSTM(32)(Lambda(lambda x: x[:, :, 1:2])(inp))
bH = LSTM(32)(Lambda(lambda x: x[:, :, 2:3])(inp))

x = Concatenate()([bE, bN, bH])
x = Dense(32, activation="relu")(x)
out = Dense(1)(x)

model = Model(inp, out)
model.compile(optimizer="adam", loss=tf.keras.losses.Huber(0.2))

# TRAIN
with st.spinner("Training PLSTM..."):
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=300,
        batch_size=8,
        callbacks=[EarlyStopping(patience=30, restore_best_weights=True)],
        verbose=0
    )

st.success("Training selesai.")

# EVALUASI 2024 (TEST)
y_pred = model.predict(X_test, verbose=0)

y_true_m = scaler_H.inverse_transform(y_test)
y_pred_m = scaler_H.inverse_transform(y_pred)

mse = mean_squared_error(y_true_m, y_pred_m)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_m, y_pred_m)

st.subheader("Evaluasi Test (2024)")
st.write(pd.DataFrame({
    "MSE": [mse],
    "RMSE": [rmse],
    "MAE": [mae]
}))

# PREDICTION 2025–2029
last_seq = X[-1]
future = []
cur = last_seq.copy()

for _ in range(60):
    p = model.predict(cur.reshape(1, TIME_STEPS, 3), verbose=0)[0, 0]
    future.append(p)
    cur = np.roll(cur, -1, axis=0)
    cur[-1, 2] = p

future_m = scaler_H.inverse_transform(np.array(future).reshape(-1, 1)).ravel()

future_months = pd.date_range(start=monthly_delta.index[-1] + pd.DateOffset(months=1),
                              periods=60, freq="MS")

prediction_df = pd.DataFrame({
    "Predicted_dH_m": future_m,
    "Predicted_dH_mm": future_m * 1000
}, index=future_months)

st.subheader("prediction 2025–2029")
st.dataframe(prediction_df.head(12))

Overwriting app.py


In [ ]:
!ls

app.py	deploy	requirements.txt


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.187.61:8501

  Stopping...
